In [369]:
import pandas as pd
import itertools

In [370]:
file_path = r'C:\Users\20127\Downloads\Horizontal_Format.xlsx'
df = pd.read_excel(file_path)

In [371]:
df.dtypes

TiD       int64
items    object
dtype: object

In [372]:
df

,TiD,items
0,1,"M,O,N,K,E,Y"
1,2,"D,O,N,K,E,Y"
2,3,"M,A,K,E"
3,4,"M,U,C,K,Y"
4,5,"C,O,O,K,I,E"


In [373]:
# import pandas as pd

# num_of_transactions = int(input("Enter the number of transactions: "))


# transactions = []


# for i in range(1, num_of_transactions + 1):
#     tid = f'{i}' 
#     items_input = input(f"Enter items for transaction {tid} (comma-separated): ")
#     items = [item.strip() for item in items_input.split(',')]
#     transactions.append({'TiD': tid, 'items': ', '.join(items)})


# transactions_df = pd.DataFrame(transactions, columns=['TiD', 'items'])


# transactions_df.to_excel(r'C:\Users\20127\Downloads\vert.xlsx', index=False)


In [374]:
#df=pd.DataFrame(transactions, columns=['TiD', 'items'])

In [375]:
#df.to_excel(r'C:\Users\20127\Downloads\vert.xlsx', index=True)

In [376]:
if df.columns[0]=="TiD" and df.columns[1]=="items":
    data_list=[]
    for index,row in df.iterrows():
        TiD = [row['TiD']] * len(row['items'])
        items = row['items']
        items = [item for item in items if item != ","]
        data_list.extend(zip(items, TiD))
    df_vertical = pd.DataFrame(data_list, columns=['itemset', 'TiD_set'])    
    df_vertical = df_vertical.sort_values(by='itemset')
    df_vertical = df_vertical.groupby('itemset')['TiD_set'].agg(list).reset_index()                              
    df_vertical['TiD_set'] = df_vertical['TiD_set'].apply(lambda x: sorted(set(x)))
else:
    df_vertical=df  

In [377]:
df_vertical

,itemset,TiD_set
0,A,[3]
1,C,"[4, 5]"
2,D,[2]
3,E,"[1, 2, 3, 5]"
4,I,[5]
5,K,"[1, 2, 3, 4, 5]"
6,M,"[1, 3, 4]"
7,N,"[1, 2]"
8,O,"[1, 2, 5]"
9,U,[4]


In [378]:
df_vertical.dtypes

itemset    object
TiD_set    object
dtype: object

In [379]:
def join_items(item1, item2, transactions1, transactions2):
    mutual_transactions = list(set(transactions1) & set(transactions2))
    joining_items = list(set(item1.split(',')) | set(item2.split(',')))
    joining_items.sort()
    return {'itemset': ','.join(joining_items), 'TiD_set': mutual_transactions}

In [380]:
def checkingsupport(df, min_support):
    L = []
    for index, row in df.iterrows():
        each_item_support = len(row['TiD_set'])
        if each_item_support >= min_support:
            TiD = row['TiD_set']
            items = row['itemset']
            L.append({'itemset': items, 'TiD_set': TiD})

    return pd.DataFrame(L, columns=['itemset', 'TiD_set'])

In [381]:
def join_withthesamestart(item1, item2, transactions1, transactions2):
    if not item1 or not item2:
        return None

    prefix_len = len(item1) - 1
    prefix1 = item1[:prefix_len] if item1 else ""
    prefix2 = item2[:prefix_len] if item2 else ""

    if prefix1 == prefix2:
        joining_items = list(set(item1.split(',')) | set(item2.split(',')))
        joining_items.sort()
        common_transactions = list(set(transactions1) & set(transactions2))
        return {'itemset': ','.join(joining_items), 'TiD_set': common_transactions}
    else:
        return None

In [382]:
def generate_candidate_dataset(item_dataset):
    C = []
    for (index1, row1), (index2, row2) in itertools.combinations(item_dataset.iterrows(), 2):
        item1, transactions1 = row1['itemset'], row1['TiD_set']
        item2, transactions2 = row2['itemset'], row2['TiD_set']
        
        if len(item1) ==1 and len(item2) ==1 :
            result = join_items(item1, item2, transactions1, transactions2)
            
        else:
            result = join_withthesamestart(item1, item2, transactions1, transactions2)
          
            
        if result:
            C.append(result)
    return pd.DataFrame(C, columns=['itemset', 'TiD_set'])

In [383]:
def ECLAT(df, min_support):
    frequent_itemsets = []
    frequent_1_itemsets = checkingsupport(df, min_support)
    frequent_itemsets.append(frequent_1_itemsets)
    candidates=[]

    k = 2

    while True:
        candidate_itemsets = generate_candidate_dataset(frequent_itemsets[k-2])
        candidates.append(candidate_itemsets)
        frequent_k_itemsets = checkingsupport(candidate_itemsets, min_support)
        
        if frequent_k_itemsets.empty:
            break
        frequent_itemsets.append(frequent_k_itemsets)
        k += 1
     
    result = pd.concat(frequent_itemsets, ignore_index=True) # 5lethom t7t b3d 3shan aloop ashl bdl el NaN ely kant bttl3
    candidates_result= pd.concat(candidates, ignore_index=True)


    return result,candidates_result

In [384]:
result,candidates_result = ECLAT(df_vertical, 3)

In [385]:
candidates_result

,itemset,TiD_set
0,"E,K","[1, 2, 3, 5]"
1,"E,M","[1, 3]"
2,"E,O","[1, 2, 5]"
3,"E,Y","[1, 2]"
4,"K,M","[1, 3, 4]"
5,"K,O","[1, 2, 5]"
6,"K,Y","[1, 2, 4]"
7,"M,O",[1]
8,"M,Y","[1, 4]"
9,"O,Y","[1, 2]"


In [386]:
result

,itemset,TiD_set
0,E,"[1, 2, 3, 5]"
1,K,"[1, 2, 3, 4, 5]"
2,M,"[1, 3, 4]"
3,O,"[1, 2, 5]"
4,Y,"[1, 2, 4]"
5,"E,K","[1, 2, 3, 5]"
6,"E,O","[1, 2, 5]"
7,"K,M","[1, 3, 4]"
8,"K,O","[1, 2, 5]"
9,"K,Y","[1, 2, 4]"


In [387]:
from itertools import product

def generate_rules(min_confidence):
    confidence_list = []

    for i in range(len(result)):
        itemset_i_raw = result['itemset'][i]

        if isinstance(itemset_i_raw, str):
            itemset_i = itemset_i_raw.split(',')
        else:
            itemset_i = itemset_i_raw

        for index, row in result.iterrows():
            itemset = row['itemset'].split(',')
            tid_set = set(row['TiD_set'])

            if set(itemset_i) & set(itemset):
                continue

            if isinstance(itemset_i, list):
                for item_combination in product(itemset_i, repeat=len(itemset_i)):
                    item1 = ','.join(item_combination)

                    if not result[result['itemset'] == item1].empty:
                        item1_tid_set = set(result[result['itemset'] == item1]['TiD_set'].iloc[0])
                        common_tid_set = tid_set.intersection(item1_tid_set)

                        if len(item1_tid_set) > 0:
                            confidence = len(common_tid_set) / len(item1_tid_set)
                        else:
                            confidence = 0

                        strong_rule = 'yes' if confidence >= min_confidence else 'no'

                        confidence_list.append({
                            'Item1': item1,
                            'Item2': ','.join(itemset),
                            'Confidence': confidence,
                            'Strong': strong_rule
                        })
            else:
                item1 = itemset_i

                if not result[result['itemset'] == item1].empty:
                    item1_tid_set = set(result[result['itemset'] == item1]['TiD_set'].iloc[0])
                    common_tid_set = tid_set.intersection(item1_tid_set)

                    if len(item1_tid_set) > 0:
                        confidence = len(common_tid_set) / len(item1_tid_set)
                    else:
                        confidence = 0

                    strong_rule = 'yes' if confidence >= min_confidence else 'no'

                    confidence_list.append({
                        'Item1': item1,
                        'Item2': ','.join(itemset),
                        'Confidence': confidence,
                        'Strong': strong_rule
                    })

    confidence_df = pd.DataFrame(confidence_list)
    return confidence_df


In [388]:
rules_df=generate_rules(0.66)

In [389]:
rules_df

,Item1,Item2,Confidence,Strong
0,E,K,1.000000,yes
1,E,M,0.500000,no
2,E,O,0.750000,yes
3,E,Y,0.500000,no
4,E,"K,M",0.500000,no
5,E,"K,O",0.750000,yes
6,E,"K,Y",0.500000,no
7,K,E,0.800000,yes
8,K,M,0.600000,no
9,K,O,0.600000,no


In [390]:
import pandas as pd

def generate_lift(result):
    lift_list = []

    for i in range(len(result)):
        itemset_i_raw = result['itemset'][i]

        if isinstance(itemset_i_raw, str):
            itemset_i = itemset_i_raw.split(',')
        else:
            itemset_i = itemset_i_raw

        for j in range(i + 1, len(result)):
            itemset_j_raw = result['itemset'][j]

            if isinstance(itemset_j_raw, str):
                itemset_j = itemset_j_raw.split(',')
            else:
                itemset_j = itemset_j_raw

            if set(itemset_i) & set(itemset_j):
                continue  

            tid_set = set(result['TiD_set'][i])
            item1 = ','.join(map(str, itemset_i))
            item2 = ','.join(map(str, itemset_j))

            common_tid_set = tid_set.intersection(set(result['TiD_set'][j]))

            support_item1 = len(tid_set) / len(result)
            support_item2 = len(result['TiD_set'][j]) / len(result)
            support_both = len(common_tid_set) / len(result)

            if support_item1 * support_item2 > 0:
                lift = support_both / (support_item1 * support_item2)
            else:
                lift = 0
            relation = 'independent' if lift == 1 else '+ve correlated' if lift > 1 else '-ve correlated'
            lift_list.append({
                'Item1': item1,
                'Item2': item2,
                'Lift': lift,
                'Relation': relation
            })

    return pd.DataFrame(lift_list)


In [391]:
generate_lift(result)

,Item1,Item2,Lift,Relation
0,E,K,2.200000,+ve correlated
1,E,M,1.833333,+ve correlated
2,E,O,2.750000,+ve correlated
3,E,Y,1.833333,+ve correlated
4,E,"K,M",1.833333,+ve correlated
5,E,"K,O",2.750000,+ve correlated
6,E,"K,Y",1.833333,+ve correlated
7,K,M,2.200000,+ve correlated
8,K,O,2.200000,+ve correlated
9,K,Y,2.200000,+ve correlated
